In [6]:
from py_modules import gcs_helper

# V = Validation folder
gcsV = gcs_helper.gcsh('devandtest')

In [7]:
bkt = 'samartest'
fld = 'templates/validation'

P021 = '0' 
P001 = '1'
P019 = '2' 
P022 = '3' 
P020 = '4' 
P002 = '5' 

# squarepoint
cust_sqp = '26'

fle01 = f'{fld}/{cust_sqp}-{P001}.txt'
fle02 = f'{fld}/{cust_sqp}-{P002}.txt'
fle19 = f'{fld}/{cust_sqp}-{P019}.txt'
fle20 = f'{fld}/{cust_sqp}-{P020}.txt'
fle21 = f'{fld}/{cust_sqp}-{P021}.txt'
fle22 = f'{fld}/{cust_sqp}-{P022}.txt'

sql01 = gcsV.file_get(bkt, fle01, False)
sql02 = gcsV.file_get(bkt, fle02, False)
sql19 = gcsV.file_get(bkt, fle19, False)
sql20 = gcsV.file_get(bkt, fle20, False)
sql21 = gcsV.file_get(bkt, fle21, False)
sql22 = gcsV.file_get(bkt, fle22, False)

print_len = 80

print(sql01[:print_len])
print(sql02[:print_len])
print(sql19[:print_len])
print(sql20[:print_len])
print(sql21[:print_len])
print(sql22[:print_len])

b'SELECT yoy_yield, yoy_rpk, yoy_revenue, yoy_net_revenue, yoy_yield_7day, yoy_rpk'
b'SELECT yoy_yield_7day, yoy_rpk_7day, yoy_revenue_7day, yoy_net_revenue_7day, yoy'
b'SELECT cumulative_yoy_yield, cumulative_yoy_rpk, cumulative_yoy_revenue, cumulat'
b'SELECT yoy_yield_7day, yoy_rpk_7day, yoy_revenue_7day, yoy_net_revenue_7day, cum'
b'SELECT cumulative_yoy_yield, cumulative_yoy_rpk, cumulative_yoy_revenue, cumulat'
b'SELECT yoy_yield, yoy_rpk, yoy_revenue, yoy_net_revenue, yoy_yield_7day, yoy_rpk'


In [8]:
print_len = 2000

# print(sql01.decode()[:print_len] + '\n\n')
# print(sql02.decode()[:print_len] + '\n\n')
# print(sql19.decode()[:print_len] + '\n\n')
# print(sql20.decode()[:print_len] + '\n\n')
# print(sql21.decode()[:print_len] + '\n\n')
# print(sql22.decode()[:print_len] + '\n\n')

In [9]:
import pandas as pd
from io import StringIO

In [10]:
bkt = 'd-dat-data-eng-dev'
fle = '2020-08-24-26-1.csv'

# https://gcsfs.readthedocs.io/en/latest/#credentials doesn't work
# df_squarepoint = pd.read_excel(f'gcs://{bkt}/{csv_squarepoint}', storage_options={"token": "anon"})

csv_squarepoint = gcsV.file_get(bkt, fle, False)

In [11]:
data = StringIO(csv_squarepoint.decode())

# don't think can be split from StringIO action above - once only stream?
df_csv = pd.read_csv(data, nrows=2000, header = [5,6])

In [2]:
from py_modules import bq_helper

bqh = bq_helper.bqh('devandtest')

In [3]:
# prj_ds = 'skytra-benchmark-rnd.products_m'
prj_ds = 'skytra-benchmark-devandtest.scratch_PaD'

In [4]:
"""
No sa permissions to RnD so need to copy to DevTest/other to both a) not save credentials in Jupyter Lab b) use sa credentials/Jupyter

bq cp skytra-benchmark-rnd:products_IN1139.P001_Revenue_per_region_pair skytra-benchmark-devandtest:scratch_PaD.P001_Revenue_per_region_pair
"""

'\nbq cp skytra-benchmark-rnd:products_IN1139.P001_Revenue_per_region_pair skytra-benchmark-devandtest:scratch_PaD.P001_Revenue_per_region_pair\n'

In [12]:
Ptbl = 'P0001'
Vtbl = f'vw_{Ptbl}_Pivot'

sql01_fin = f"""
SELECT dt_of_issue, sum(yoy_rpk_7day) as yoy_rpk_7day
FROM {prj_ds}.P001_Revenue_per_region_pair 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND region_pair IN ('North America-North America') 
GROUP BY 1
ORDER BY 1
"""


sql = sql01_fin
csv_cols = ['Product Code', '99NANAWEKALLDRPY']


# having manually derived _fin(al) sql from automatically downloaded validation sql
#  run the qry against the active GoogleSDK project
df01 = bqh.run_qry(sql)
# df01.head()

# get the equivalent column(s) from the CSV file
df_csv_sub01 = df_csv[csv_cols]
df_csv_sub01.columns = df_csv_sub01.columns.droplevel()
# df_csv_sub01.head()

# merge and visually compare prior to auto-compare
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
dfm = pd.merge(df01, df_csv_sub01, how = 'outer', left_index = True, right_index = True)
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [13]:
Ptbl = 'P0001'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')
    # dfm[~dfm['Equal']==False & dfm['VAR'].abs() > .001].head(1000)
    # TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]
print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
display(dfm)



SELECT dt_of_issue, sum(yoy_rpk_7day) as yoy_rpk_7day
FROM skytra-benchmark-devandtest.scratch_PaD.P001_Revenue_per_region_pair 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND region_pair IN ('North America-North America') 
GROUP BY 1
ORDER BY 1

 **P table P0001** sql to compare to Airtyx CSV columns... 

['Product Code', '99NANAWEKALLDRPY']

 **START VARIANCES *************** 
 *********************************
Empty DataFrame
Columns: [dt_of_issue, yoy_rpk_7day, Product Date, Cumulative volume sold (RPK) - % change YoY, Var, Equal]
Index: []

 **END VARIANCES ***************** 
 *********************************

 BASIC STATS 

dt_of_issue.min() = 2019-01-01
dt_of_issue.max() = 2019-12-31
dt_of_issue                                    365
yoy_rpk_7day                                   365
Product Date                                   365
Cumulative volume sold (RPK) - % change YoY    365
Var                                            365
Equal               

,dt_of_issue,yoy_rpk_7day,Product Date,Cumulative volume sold (RPK) - % change YoY,Var,Equal
0,2019-01-01,0.027917679,2019-01-01,0.027918,0.000000e+00,True
1,2019-01-02,0.034785132,2019-01-02,0.034785,0.000000e+00,True
2,2019-01-03,0.026887988,2019-01-03,0.026888,3.469447e-18,False
3,2019-01-04,0.029920895,2019-01-04,0.029921,0.000000e+00,True
4,2019-01-05,-0.007237113,2019-01-05,-0.007237,8.673617e-19,False
...,...,...,...,...,...,...
360,2019-12-27,0.03891177,2019-12-27,0.038912,0.000000e+00,True
361,2019-12-28,0.056607976,2019-12-28,0.056608,0.000000e+00,True
362,2019-12-29,0.055012132,2019-12-29,0.055012,-6.938894e-18,False
363,2019-12-30,0.062972099,2019-12-30,0.062972,0.000000e+00,True


In [ ]:

sql02_fin = f"""
SELECT dt_of_issue, sum(yoy_revenue) as yoy_revenue
FROM {prj_ds}.P002_Revenue_yield_per_region_pair_airline 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND flight_op_carrier IN ('B6') AND region_pair IN ('Latin America-North America')
GROUP BY 1
ORDER BY 1
"""

Ptbl = 'P0002'
sql = sql02_fin
csv_cols = ['Product Code', 'B6LANAALLALLDGRY']


# having manually derived _fin(al) sql from automatically downloaded validation sql
#  run the qry against the active GoogleSDK project
df02 = bqh.run_qry(sql)
# df02.head()

# get the equivalent column(s) from the CSV file
df_csv_sub02 = df_csv[csv_cols]
df_csv_sub02.columns = df_csv_sub02.columns.droplevel()
# df_csv_sub02.head()

# merge and visually compare prior to auto-compare
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
dfm = pd.merge(df02, df_csv_sub02, how = 'outer', left_index = True, right_index = True)
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [ ]:
Ptbl = 'P0002'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')

print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
print(dfm)


In [ ]:

sql19_fin = f"""
SELECT dt_of_issue, sum(cumulative_yoy_revenue) as cumulative_yoy_revenue 
FROM {prj_ds}.P019_S_curve_months_per_airline 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND op_carrier IN ('UA') AND code_delivery IN ('M092')
GROUP BY 1
ORDER BY 1
"""

Ptbl = 'P0019'
sql = sql19_fin
csv_cols = ['Product Code', 'UAWWWWALLM092GRY']

#  run the qry against the active GoogleSDK project
df19 = bqh.run_qry(sql)
df19 = df19[df19.cumulative_yoy_revenue.notnull()]

# get the equivalent column(s) from the CSV file
df_csv_sub19 = df_csv[csv_cols]
df_csv_sub19.columns = df_csv_sub19.columns.droplevel()
df_csv_sub19.columns = ['product_date', df_csv_sub19.iloc[:,1].name]

# convert from object to datetime to allow merge by name - needed because cannot merge by index for Delivery Periods
df_csv_sub19.product_date = pd.to_datetime(df_csv_sub19.product_date)

# merge 
dfm = pd.merge(df19, df_csv_sub19, how='inner', on = None, left_on = 'dt_of_issue', right_on = 'product_date')
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [ ]:
Ptbl = 'P0019'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')

print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
print(dfm)


In [ ]:
df20.head()

In [ ]:

sql20_fin = f"""
SELECT dt_of_issue, sum(yoy_yield_7day) as cumulative_yoy_yield_7day 
FROM {prj_ds}.P020_S_curve_months_per_airline_region 
WHERE dt_of_issue >= '2020-01-01' and dt_of_issue <= '2020-12-31' AND op_carrier IN ('B6') AND region_pair IN ('Middle East-North America') AND code_delivery IN ('M073')
GROUP BY 1
ORDER BY 1
"""

Ptbl = 'P0020'
sql = sql20_fin
csv_cols = ['Product Code', 'B6MENAWEKM073YYO']

#  run the qry against the active GoogleSDK project
df20 = bqh.run_qry(sql)
df20 = df20[df20.cumulative_yoy_revenue.notnull()]

# get the equivalent column(s) from the CSV file
df_csv_sub20 = df_csv[csv_cols]
df_csv_sub20.columns = df_csv_sub20.columns.droplevel()
df_csv_sub20.columns = ['product_date', df_csv_sub20.iloc[:,1].name]

# convert from object to datetime to allow merge by name - needed because cannot merge by index for Delivery Periods
df_csv_sub20.product_date = pd.to_datetime(df_csv_sub20.product_date)

# merge 
dfm = pd.merge(df20, df_csv_sub20, how='inner', on = None, left_on = 'dt_of_issue', right_on = 'product_date')
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [ ]:
Ptbl = 'P0020'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')

print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
print(dfm)


In [ ]:

sql21_fin = f"""
SELECT dt_of_issue, sum(cumulative_yoy_yield) as cumulative_yoy_yield
FROM {prj_ds}.P021_S_curve_months_per_region 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND region_pair IN ('North America-North America') AND code_delivery IN ('M073') 
GROUP BY 1
ORDER BY 1
"""

Ptbl = 'P0021'
sql = sql21_fin
csv_cols = ['Product Code', '99NANAALLM073YYO']

#  run the qry against the active GoogleSDK project
df21 = bqh.run_qry(sql)
df21 = df21[df21.iloc[:,1].notnull()]
df21.dt_of_issue = pd.to_datetime(df21.dt_of_issue)

# get the equivalent column(s) from the CSV file
df_csv_sub21 = df_csv[csv_cols]
df_csv_sub21.columns = df_csv_sub21.columns.droplevel()
df_csv_sub21.columns = ['product_date', df_csv_sub21.iloc[:,1].name]

# convert from object to datetime to allow merge by name - needed because cannot merge by index for Delivery Periods
# df_csv_sub21.product_date = pd.to_datetime(df_csv_sub21.product_date)

# merge 
dfm = pd.merge(df21, df_csv_sub21, how='inner', on = None, left_on = 'dt_of_issue', right_on = 'product_date')
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])


In [ ]:
Ptbl = 'P0021'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')

print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
print(dfm)


In [ ]:

sql22_fin = f"""
SELECT dt_of_issue, sum(yoy_net_revenue_7day) as yoy_net_revenue_7day  
FROM {prj_ds}.P022_Revenue_per_airline 
WHERE dt_of_issue >= '2019-01-01' and dt_of_issue <= '2019-12-31' AND op_carrier IN ('DL') GROUP BY 1
ORDER BY 1
"""

Ptbl = 'P0022'
sql = sql22_fin
csv_cols = ['Product Code', 'DLWWWWWEKALLDNRY']

#  run the qry against the active GoogleSDK project
df22 = bqh.run_qry(sql)
# .notnull() filter not needed because no Delivery Month
df22.dt_of_issue = pd.to_datetime(df22.dt_of_issue)

# get the equivalent column(s) from the CSV file
df_csv_sub22 = df_csv[csv_cols]
df_csv_sub22.columns = df_csv_sub22.columns.droplevel()
df_csv_sub22.columns = ['product_date', df_csv_sub22.iloc[:,1].name]

# error A value is trying to be set on a copy of a slice from a DataFrame.
# HOWEVER, bogus error, works anyway, code_kata nb on personal laptop has the method to switch this off
df_csv_sub22.product_date = pd.to_datetime(df_csv_sub22.product_date)

# merge could be done using index because no Delivery Month, but in case outer join issues, preferable to join by date
dfm = pd.merge(df22, df_csv_sub22, how='outer', on = None, left_on = 'dt_of_issue', right_on = 'product_date')
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [ ]:
dfm = pd.merge(df22, df_csv_sub22, how='outer', on = None, left_on = 'dt_of_issue', right_on = 'product_date')
dfm['Var'] = dfm.iloc[:,1].astype(float) - dfm.iloc[:,3]
dfm['Equal'] = dfm.iloc[:,1].astype(float).eq(dfm.iloc[:,3])

In [ ]:
Ptbl = 'P0022'
print(sql + f'\n **P table {Ptbl}** sql to compare to Airtyx CSV columns... \n')
print(csv_cols)

print('\n **START VARIANCES *************** \n *********************************')

print(dfm[dfm['Var'].abs() > .00001])
print('\n **END VARIANCES ***************** \n *********************************')

print('\n BASIC STATS \n')
print( 'dt_of_issue.min() = ' + str(dfm.dt_of_issue.min()) )
print( 'dt_of_issue.max() = ' + str(dfm.dt_of_issue.max()) )
print(dfm.count())
print('\n DATAFRAME SUMMARY \n')
print(dfm)
